In [3]:
import argparse
import numpy as np
import os.path
from tqdm import tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.externals import joblib

from osgeo import gdal

from util.geometry import MapGeometry
from util.load_data import read_records, get_map_from_i_j_to_example_index

In [4]:
nightlights_raster = 'F182013.v4c_web.stable_lights.avg_vis.tif'

In [5]:
raster_dataset = gdal.Open(nightlights_raster, gdal.GA_ReadOnly)

In [6]:
raster_dataset

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f8e952e5900> >

In [10]:
# get project coordination
proj = raster_dataset.GetProjectionRef()
bands_data = []

In [11]:
# Loop through all raster bands
for b in range(1, raster_dataset.RasterCount + 1):
    band = raster_dataset.GetRasterBand(b)
    bands_data.append(band.ReadAsArray())
    no_data_value = band.GetNoDataValue()
bands_data = np.dstack(bands_data)
rows, cols, n_bands = bands_data.shape

In [15]:
n_bands

1

In [16]:
# Get the metadata of the raster
geo_transform = raster_dataset.GetGeoTransform()
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = geo_transform

In [17]:
# Get location of each pixel
x_size = 1.0 / int(round(1 / float(x_size)))
y_size = - x_size
y_index = np.arange(bands_data.shape[0])
x_index = np.arange(bands_data.shape[1])
top_left_x_coords = upper_left_x + x_index * x_size
top_left_y_coords = upper_left_y + y_index * y_size

In [18]:
top_left_y_coords

array([ 75.00416667,  74.99583333,  74.9875    , ..., -64.97916667,
       -64.9875    , -64.99583333])

In [19]:
# Add half of the cell size to get the centroid of the cell
centroid_x_coords = top_left_x_coords + (x_size / 2)
centroid_y_coords = top_left_y_coords + (y_size / 2)

self.x_size = x_size
self.top_left_x_coords = top_left_x_coords
self.top_left_y_coords = top_left_y_coords
self.centroid_x_coords = centroid_x_coords
self.centroid_y_coords = centroid_y_coords